### Import libraries



In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional, Embedding, Flatten
from keras.optimizers import SGD
import math
from sklearn.metrics import mean_squared_error
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from langdetect import detect
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam



2023-12-19 14:25:14.799391: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-19 14:25:15.079765: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-19 14:25:15.079837: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-19 14:25:15.133120: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-19 14:25:15.241105: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-19 14:25:15.242447: I tensorflow/core/platform/cpu_feature_guard.cc:1

### Dataset

In [2]:
dataset = pd.read_csv('preprocessed_train.csv')

# give a description of the data
print(dataset.describe())

         Unnamed: 0        rating  preprocessed_review_length
count  31246.000000  31246.000000                31246.000000
mean   15907.414485      0.238174                   41.354733
std     9259.086027      0.946370                   51.817764
min        0.000000     -1.000000                    0.000000
25%     7863.250000     -1.000000                   13.000000
50%    15854.500000      1.000000                   25.000000
75%    23924.750000      1.000000                   50.000000
max    32035.000000      1.000000                 1528.000000


In [3]:

df = pd.read_csv('preprocessed_train.csv')


# Drop rows with NaN values in the 'preprocessed_review' column
df = df.dropna(subset=['preprocessed_review'])

# Convert ratings to one-hot encoded labels
labels = to_categorical(df['rating'] + 1)  # Adding 1 to convert -1, 0, 1 to 0, 1, 2

# Tokenize the Arabic text
tokenizer_arabic = Tokenizer()
tokenizer_arabic.fit_on_texts(df['preprocessed_review'])
sequences_arabic = tokenizer_arabic.texts_to_sequences(df['preprocessed_review'])
padded_sequences_arabic = pad_sequences(sequences_arabic)

# Define the LSTM model with dropout layers
model = Sequential()


model.add(Embedding(input_dim=len(tokenizer_arabic.word_index) + 1, output_dim=100, input_length=padded_sequences_arabic.shape[1]))

model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2)) # Adding dropout to the LSTM layer

model.add(Dense(64, activation='relu'))

model.add(Dropout(0.5))  # Adding dropout to the Dense layer

model.add(Dense(3, activation='softmax'))  # Three output nodes for negative, neutral, and positive

# Compile the model with categorical_crossentropy
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model on Arabic data
model.fit(padded_sequences_arabic, labels, epochs=5, batch_size=32, validation_split=0.2)

Epoch 1/5
773/773 [==============================] - 215s 275ms/step - loss: 0.5483 - accuracy: 0.7933 - val_loss: 0.4501 - val_accuracy: 0.8345
Epoch 2/5
773/773 [==============================] - 207s 268ms/step - loss: 0.3671 - accuracy: 0.8754 - val_loss: 0.4658 - val_accuracy: 0.8312
Epoch 3/5
773/773 [==============================] - 205s 266ms/step - loss: 0.2829 - accuracy: 0.9039 - val_loss: 0.5370 - val_accuracy: 0.8241
Epoch 4/5
773/773 [==============================] - 206s 267ms/step - loss: 0.2288 - accuracy: 0.9215 - val_loss: 0.5760 - val_accuracy: 0.8029
Epoch 5/5
773/773 [==============================] - 218s 282ms/step - loss: 0.1954 - accuracy: 0.9348 - val_loss: 0.6405 - val_accuracy: 0.8040


In [4]:
# show the accuracy of the model
loss, accuracy = model.evaluate(padded_sequences_arabic, labels, verbose=False)

print("Training Accuracy: {:.4f}".format(accuracy))




Training Accuracy: 0.9239


In [7]:
# read the test data
test_data = pd.read_csv('preprocessed_test.csv')



# Tokenize the preprocessed reviews in the test data
sequences_test = tokenizer_arabic.texts_to_sequences(test_data['preprocessed_review'])



padded_sequences_test = pad_sequences(sequences_test, maxlen=padded_sequences_arabic.shape[1])

# Predict the sentiment analysis using the trained model
predictions = model.predict(padded_sequences_test)
predicted_ratings = np.argmax(predictions, axis=1) - 1

# Add the predicted ratings as a new column in the test data
test_data['rating'] = predicted_ratings

# Save the test data with the predicted ratings as a CSV file
test_data.to_csv('predicted_test.csv', index=False)


31/31 [==============================] - 2s 72ms/step
